In [ ]:
import subprocess

subprocess.run(["python", "-m", "run_parallel_experiments"])

# IF YOU WANT TO INSPECT THE OUTPUTS OF THE EXPERIMENTS IN REAL TIME:
# strace -p857 -s9999 -e write 2>&1 | grep -o '".\+[^"]"' 
# where 857 is the PID of the python process

Process agents started
Process tasks started
Process tasks_frequency started
Process starts started
Process goals started


ic| f": ('Simulation Start:
        '
         '\tMap Name: warehouse_3_3
        '
         '\tAgent number: 24
        '
         '\tPickup number: 18
        '
         '\tGoal number: 14
        '
         '\tTask number: 100
        '
         '\tTask frequency: 0.20
        '
         '\tTask Distribution Update Frequency: 15
        '
         '\tLearning: False')
ic| f": ('Simulation Start:
        '
         '\tMap Name: warehouse_3_3
        '
         '\tAgent number: 24
        '
         '\tPickup number: 5
        '
         '\tGoal number: 14
        '
         '\tTask number: 100
        '
         '\tTask frequency: 1.00
        '
         '\tTask Distribution Update Frequency: 15
        '
         '\tLearning: False')
ic| f": ('Simulation Start:
        '
         '\tMap Name: warehouse_3_3
        '
         '\tAgent number: 24
        '
         '\tPickup number: 18
        '
         '\tGoal number: 14
        '
         '\tTask number: 100
        '
         '\tT

In [1]:
from IPython.display import Markdown, display
from wand.image import Image as WImage
import matplotlib.pyplot as plt
import json
from stats_visualizer import StatsVisualizer
import os
from glob import glob

%matplotlib inline

plt.style.use('seaborn-v0_8-dark-palette')

def printmd(string):
    display(Markdown(string))


In [2]:
result_file_names = glob(os.path.join('ResultsJsons', '*.json'))

for result_file_name in result_file_names:
    with open(result_file_name, "r") as f:
        results = json.load(f)
    maps = results["maps"]

    possible_number_of_tasks = results["tasks_num"]
    possible_number_of_task_freq = results["tasks_frequency"]
    possible_number_of_td_updates = results["task_distr_update_num"]

    result_file_name = str(result_file_name.split('/')[-1]).split('.')[0]

    printmd(f"## Saving plots for file: {result_file_name} ...")

    stats_visualizer = StatsVisualizer(maps, possible_number_of_tasks, possible_number_of_task_freq, possible_number_of_td_updates, save=True)

    stats_visualizer.show_all_metrics()

printmd("Finished.")

## Saving plots for file: results_05_10_2024__08_27_41 ...

## Saving plots for file: results_05_10_2024__04_09_41 ...

## Saving plots for file: results_05_10_2024__05_06_38 ...

## Saving plots for file: results_05_10_2024__07_51_36 ...

## Saving plots for file: results_05_10_2024__06_42_04 ...

## Saving plots for file: results_06_10_2024__05_34_20 ...

Finished.

In [ ]:
# Display map images

for image_path in glob(os.path.join('../maps_pdfs', '*.pdf')):
    printmd(image_path.split('/')[-1])
    display(WImage(filename=image_path))